In [1]:
from os import listdir
from pprint import pprint

import pdfplumber
import pandas as pd

from model.columns_pdf import ColumnsPdf
from model.pdf import Pdf

In [2]:
def read_pdf(path: str):
    with pdfplumber.open(path) as pdf:
        texts = []
        table = []
        for page in pdf.pages:
            extracted = page.extract_text()
            if extracted is None:
                continue
            texts.append(list(filter(lambda i: i, extracted.split("\n"))))
            table += page.extract_table(table_settings={
                "vertical_strategy": "lines",
                "horizontal_strategy": "text",
                "snap_tolerance": 2,
            }) or []
    return texts, table

# DEBUG
texts, table = read_pdf("./pdfs/Barbosa ENEL_SP loja 19 abr21.pdf")
pd.DataFrame(
    data=table
)

,0,1,2,3,4,5,6,7,8,9
0,"PERDA DE TRANSFORMAÇÃO - 2,5%",,,,,,,,,
1,MEDIDOR N° 15793939,,,,,,,,,
2,"CONST. POTENCIA 0,06400",,,,,,,,,
3,"CONST. ATIVO 0,01600",,,,,,,,,
4,"CONST. REATIVO 0,01600",,,,,,,,,
5,CONSUMO PONTA AM,7.810.576,8.095.657,"4.561,3",,,,,,
6,CONSUMO FORA PONTA CAPACIT. AM,15.999.519,16.608.367,"9.741,6",,,,,,
7,CONSUMO FORA PONTA INDUTIVO AM,57.130.068,59.341.926,"35.389,7",,,,,,
8,DEMANDA PONTA,1.603,1.383,"88,5",,,,,,
9,DEMANDA FORA PONTA CAPACITIVA,1.355,1.157,"74,0",,,,,,


In [3]:
def convert_table(texts: list, table: list):
    list_lines = []
    list_exists = []
    for line in table:
        for j, text in enumerate(texts):
            if line[0] in text:
                if j in list_exists:
                    continue
                else:
                    list_exists.append(j)
                
                columns_pdf = ColumnsPdf()
                columns_pdf.cci = text.split()[0] if text.split()[0].isdigit() else ""
                columns_pdf.descricao = line[0]
                columns_pdf.leitura_alterior = line[1]
                columns_pdf.leitura_atual = line[2]
                columns_pdf.registrado = line[3]
                columns_pdf.faturado = line[4]
                columns_pdf.tarifa_com_icms = line[5]
                columns_pdf.base_icms = line[6]
                columns_pdf.aliq_icms = line[7]
                columns_pdf.icms = line[8]
                columns_pdf.valor = line[9]
                columns_pdf.tarifa_sem_impostos = text.split()[-1] if line[-1] and text.split()[-1] != line[-1] else ""
                list_lines.append(columns_pdf)

                if table[-1] == line:
                    columns_pdf = ColumnsPdf()
                    last_line = texts[j+1].split()
                    columns_pdf.cci = last_line[0]
                    columns_pdf.descricao = " ".join(last_line[1:-1])
                    columns_pdf.valor = last_line[-1]
                    list_lines.append(columns_pdf)

                break
    return list_lines

# DEBUG
# TODO
# convert_table(
#     ['0602	DEMANDA PONTA C/ DESCONTO				638,4	14,14247	9.028,55	18%	1.625,14	9.028,55	11,59682',
#     '0602	DEMANDA PONTA C/ DESCONTO				618,4	15,14247	8.028,55	18%	1.325,14	9.023,55	12,59682',
#     '0807	COSIP - SÃO PAULO - MUNICIPAL									30,47	', 'top1', 'top2'],
#     [
#         ['DEMANDA PONTA C/ DESCONTO', '', '', '', '618,4', '', '14,14247', '', '', '9.028,55', '', '18%	1.325,14', '', '9.023,55', '', '12,59682']
#         ['DEMANDA PONTA C/ DESCONTO', '', '', '', '618,4', '', '15,14247', '', '', '8.028,55', '', '18%	1.325,14', '', '9.023,55', '', '12,59682']
#     ]
#     )

In [4]:
def convert(texts: list, table: list):
    pdf = Pdf()

    first_page = texts[0]
    sec_page = texts[1]

    numbers = first_page[2].split(' ')
    pdf.n_instalacao, pdf.n_cliente, pdf.n_medidor = numbers[0], numbers[1], numbers[2]
    pdf.cnpj = first_page[3].split('CPF/CNPJ:')[1].split(' ')[1]
    pdf.endereco = first_page[6]
    for text in first_page:
        if 'CEP' in text:
            pdf.cep = text.split(' ')[1]
            break
    
    pdf.data_emissao = " ".join(sec_page[1].split()[0:3])
    pdf.conta_referente_a = " ".join(sec_page[1].split()[3:5])
    pdf.vencimento = " ".join(sec_page[1].split()[5:8])

    pdf.tabela = convert_table(first_page, table)

    return pdf

# DEBUG
# TODO
# table.insert(33, ['PIS/PASEP (0,59%)', '', '', '', '', '', '0,00', '0%', '0,00', '77,21-']) # teste
# first_page[0].insert(54, '0699 PIS/PASEP (0,59%) 0,00 0% 0,00 77,27-') # teste

In [11]:
def full(path: str):
    texts, table = read_pdf(path)
    pdf = convert(texts, table)
    pdf.filename = path.split('/')[-1]
    return pdf

# DEBUG
example = full("./pdfs/Barbosa ENEL_SP loja 19 abr21.pdf")
kkk = pd.DataFrame(example.to_dict())
df = example.to_dataframe()
table_df = example.tabela_to_dataframe()

pd.concat([df, table_df]).to_csv('a.csv')
# kkk.to_csv('a.csv')
# example.tabela_to_dataframe()

Index(['filename', 'n_instalacao', 'n_cliente', 'n_medidor', 'cnpj',
       'endereco', 'cep', 'data_emissao', 'conta_referente_a', 'vencimento',
       'tabela'],
      dtype='object')

In [6]:
from os import listdir

extract_pdfs = []

for pdf in listdir('./pdfs'):
    extract_pdfs.append(full(f'./pdfs/{pdf}'))

KeyboardInterrupt: 

In [ ]:
extract_pdfs[0].tabela_to_dataframe()
# assert extract_pdfs[1].n_instalacao == 'MTE0011443'



,cci,descricao,leitura_alterior,leitura_atual,registrado,faturado,tarifa_com_icms,base_icms,aliq_icms,icms,valor,tarifa_sem_impostos
0,,"PERDA DE TRANSFORMAÇÃO - 2,5%",,,,,,,,,,
1,,MEDIDOR N° 15793939,,,,,,,,,,
2,,"CONST. POTENCIA 0,06400",,,,,,,,,,
3,,"CONST. ATIVO 0,01600",,,,,,,,,,
4,,"CONST. REATIVO 0,01600",,,,,,,,,,
5,,CONSUMO PONTA AM,7.810.576,8.095.657,"4.561,3",,,,,,,
6,,CONSUMO FORA PONTA CAPACIT. AM,15.999.519,16.608.367,"9.741,6",,,,,,,
7,,CONSUMO FORA PONTA INDUTIVO AM,57.130.068,59.341.926,"35.389,7",,,,,,,
8,,DEMANDA PONTA,1.603,1.383,"88,5",,,,,,,
9,,DEMANDA FORA PONTA CAPACITIVA,1.355,1.157,"74,0",,,,,,,


# TODO:
- Adicionar `Data Emissão`;
- Adicionar `Conta Referente a`;
- Adicionar `Vencimento`;